In [3]:
#!pip install pylint gymnasium stable_baselines3 shimmy

In [1]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import gymnasium as gym
import subprocess
import torch
import os
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.logger import configure
from transformers import AutoTokenizer, AutoModel

In [3]:
# Verificar o uso da GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Inicialização de tokenizer e modelo CodeBERT
# tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
# model_codebert = AutoModel.from_pretrained("microsoft/codebert-base").to(device)

# # Pipeline de classificação para o feedback
# feedback_classifier = pipeline('sentiment-analysis', model='roberta-base')

# # Modelos LLM para Codificador e Revisor
# codificador_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=0 if torch.cuda.is_available() else -1)
# revisor_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=0 if torch.cuda.is_available() else -1)

# Configuração do logger do TensorBoard
# new_logger = configure('./tensorboard_logs/', ["stdout", "tensorboard"])

device

# device = "cpu"
# device

device(type='cuda')

# Função para Embeddings

In [5]:
# Definir o modelo CodeBERT para embeddings de código
class CodeBERTEmbeddingModel:
    def __init__(self):
        # Carregar o modelo e o tokenizer do CodeBERT
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
        self.model = AutoModel.from_pretrained("microsoft/codebert-base")

    def embed(self, codigo):
        """
        Gera os embeddings do código usando CodeBERT.
        """
        inputs = self.tokenizer(codigo, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Retorna os embeddings do último hidden state
        return outputs.last_hidden_state.mean(dim=1).numpy()

# Configurando ambiente

In [7]:
class CooperativeCodingEnv(gym.Env):
    def __init__(self):
        super(CooperativeCodingEnv, self).__init__()

        # Modelos LLM para Codificador e Revisor
        self.codificador_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=0 if torch.cuda.is_available() else -1)
        self.revisor_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=0 if torch.cuda.is_available() else -1)
        #self.codificador_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=-1)
        #self.revisor_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=-1)


        # Modelo de embeddings de código com CodeBERT
        self.embedding_model = CodeBERTEmbeddingModel()

        # Espaço de ações e observações
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(768,))  # Embedding de código
        self.action_space = gym.spaces.Discrete(2)  # 0: manter código, 1: refatorar código

        # Outras variáveis do ambiente
        self.codigo_atual = ""
        self.relatorios_revisores = []
        self.state = None
        self.best_code = None
        self.best_reward = -float('inf')  # Inicializa com uma recompensa muito baixa

    def reset(self, seed=None):
        # Reset do ambiente, inicializando o código e a comunicação entre os agentes
        prompt_inicial = "Você é um codificador e precisa escrever um código Python para análise temporal de vendas no arquivo Walmart.csv ."
        self.codigo_atual = self.gerar_codigo(prompt_inicial)
        self.relatorios_revisores = []
        self.state = self.obter_embedding_codigo(self.codigo_atual)
        return self.state,{}

    def step(self, action):
        # Aplica a ação escolhida e calcula as recompensas
        if len(self.relatorios_revisores) == 0:
          action = 0

        if action == 0:
            # Mantém o código atual, pede revisão
            relatorio = self.gerar_feedback(self.codigo_atual)
            self.relatorios_revisores.append(relatorio)

        elif action == 1:
            prompt_refatoracao = f"O revisor sugeriu refatorações. Aqui está o feedback: {self.relatorios_revisores[-1]}. Refatore o código."
            self.codigo_atual = self.gerar_codigo(prompt_refatoracao)

        # Verifica a qualidade e executabilidade do código
        qualidade_antiga = self.checar_qualidade_codigo(self.codigo_atual)
        executavel_antigo = self.executar_codigo(self.codigo_atual)

        # Obtém o feedback do revisor e refatora o código se necessário
        if action == 1 and len(self.relatorios_revisores) > 0:
            feedback = self.relatorios_revisores[-1]
            self.codigo_atual = self.gerar_codigo(f"Refatore o código: {feedback}")

        qualidade_nova = self.checar_qualidade_codigo(self.codigo_atual)
        executavel_novo = self.executar_codigo(self.codigo_atual)

        # Recompensa para o Codificador
        recompensa_codificador = 0
        if qualidade_nova:
            recompensa_codificador += 10
        if executavel_novo:
            recompensa_codificador += 10

        # Recompensa para o Revisor
        recompensa_revisor = 0
        if qualidade_nova and qualidade_nova > qualidade_antiga:
            recompensa_revisor += 5  # O código melhorou com base no feedback
        if executavel_novo and not executavel_antigo:
            recompensa_revisor += 5  # O código se tornou executável graças ao feedback

        # Log de recompensas
        print(f"Recompensa Codificador: {recompensa_codificador}")
        print(f"Recompensa Revisor: {recompensa_revisor}")

        # Recompensa combinada
        recompensa_total = recompensa_codificador + recompensa_revisor

        # Verifica se a nova recompensa total é melhor do que a anterior
        if recompensa_total > self.best_reward:
            self.best_reward = recompensa_total
            self.best_code = self.codigo_atual  # Armazena o melhor código

        # Atualiza o estado com o novo embedding
        self.state = self.obter_embedding_codigo(self.codigo_atual)

        # Verifica se o episódio terminou
        done = qualidade_nova and executavel_novo

        # Truncate pode ser False, exceto se houver um critério específico para truncamento
        truncated = False

        # Retorna a observação, recompensa (codificador e revisor), se o episódio terminou, se foi truncado e informações adicionais
        return self.state, recompensa_total, done, truncated, {}

    def render(self):
        # Visualização do ambiente
        print(f"Código atual:\n{self.codigo_atual}")
        print(f"Relatórios do revisor: {self.relatorios_revisores}")

    def gerar_codigo(self, prompt):
        """
        Gera o código a partir do LLM do Codificador.
        """
        response = self.codificador_llm(prompt, max_new_tokens=100, num_return_sequences=1)
        codigo = response[0]['generated_text']
        return codigo

    def gerar_feedback(self, codigo):
        """
        Gera feedback do código usando o LLM do Revisor.
        """
        prompt = f"Você é um revisor de código. Aqui está o código: {codigo}. Dê feedback detalhado sobre melhorias, erros e refatorações."
        response = self.revisor_llm(prompt, max_new_tokens=100, num_return_sequences=1)
        feedback = response[0]['generated_text']
        return feedback

    def obter_embedding_codigo(self, codigo):
        """
        Usa CodeBERT para obter a representação vetorial do código.
        """
        embedding = self.embedding_model.embed(codigo)
        return embedding

    def checar_qualidade_codigo(self, codigo):
        """
        Usa pylint para verificar a qualidade do código.
        """
        try:
            # Escreve o código para um arquivo temporário
            with open("temp_code.py", "w") as f:
                f.write(codigo)

            # Executa pylint no código
            result = subprocess.run(["pylint", "temp_code.py"], capture_output=True, text=True)

            # Imprime o relatório do pylint
            print("Relatório do pylint:")
            print(result.stdout)

            # Retorna True se o código está "limpo" o suficiente (score acima de 5, por exemplo)
            score_line = [line for line in result.stdout.splitlines() if line.startswith("Your code has been rated at")]
            if score_line:
                score = float(score_line[0].split("/")[0].split("at")[-1].strip())
                return score >= 5  # Exemplo de critério: score acima de 5
            return False

        except Exception as e:
            print(f"Erro ao verificar o código com pylint: {str(e)}")
            return False

    def executar_codigo(self, codigo):
        """
        Testa se o código gerado é executável.
        """
        try:
            # Escreve o código para um arquivo temporário
            with open("temp_code.py", "w") as f:
                f.write(codigo)

            # Executa o código em um subprocesso e verifica se roda sem erros
            result = subprocess.run(["python3", "temp_code.py"], capture_output=True, text=True)

            # Verifica a saída do código
            if result.returncode == 0:
                print("Código executado com sucesso!")
                return True
            else:
                print(f"Erro na execução do código: {result.stderr}")
                return False

        except Exception as e:
            print(f"Erro ao tentar executar o código: {str(e)}")
            return False

# Treinamento do modelo

In [ ]:
# Inicializa o ambiente
env = CooperativeCodingEnv()

# Configura o logger
new_logger = configure(folder="./logs", format_strings=["stdout", "csv", "tensorboard"])

# Configura o modelo PPO
model = PPO("MlpPolicy", env, verbose=1, device=device)
model.set_logger(new_logger)

# Treinar o modelo
model.learn(total_timesteps=10000)

# Resultados
print("\nMelhor código gerado durante o treinamento:\n")
print(env.best_code)

# Escreve o melhor código em um arquivo
with open("best_code.py", "w") as f:
    f.write(env.best_code)

# Testa o código executando-o diretamente
print("\nTestando o código perfeito:\n")
result = subprocess.run(["python3", "best_code.py"], capture_output=True, text=True)

# Verifica o resultado da execução
if result.returncode == 0:
    print("Código perfeito executado com sucesso!")
    print(f"Saída: {result.stdout}")
else:
    print(f"Erro na execução do código perfeito: {result.stderr}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Logging to ./logs


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Relatório do pylint:
************* Module temp_code
temp_code.py:7:2: E0001: Parsing failed: 'unterminated string literal (detected at line 7) (temp_code, line 7)' (syntax-error)

Erro na execução do código:   File "/content/temp_code.py", line 7
    .' Tente outra coisa.
     ^
SyntaxError: unterminated string literal (detected at line 7)

Relatório do pylint:
************* Module temp_code
temp_code.py:1:10: E0001: Parsing failed: 'invalid syntax (temp_code, line 1)' (syntax-error)

Erro na execução do código:   File "/content/temp_code.py", line 1
    Refatore o código: Você é um revisor de código. Aqui está o código: Você é um codificador e precisa escrever um código Python para análise temporal de vendas no arquivo Walmart.csv . Faça a leitura e escrita em um arquivo.
             ^
SyntaxError: invalid syntax

Recompensa Codificador: 0
Recompensa Revisor: 0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Relatório do pylint:
************* Module temp_code
temp_code.py:1:10: E0001: Parsing failed: 'invalid syntax (temp_code, line 1)' (syntax-error)

Erro na execução do código:   File "/content/temp_code.py", line 1
    Refatore o código: Você é um revisor de código. Aqui está o código: Você é um codificador e precisa escrever um código Python para análise temporal de vendas no arquivo Walmart.csv . Faça a leitura e escrita em um arquivo.
             ^
SyntaxError: invalid syntax

Relatório do pylint:
************* Module temp_code
temp_code.py:1:10: E0001: Parsing failed: 'invalid syntax (temp_code, line 1)' (syntax-error)

Erro na execução do código:   File "/content/temp_code.py", line 1
    Refatore o código: Você é um revisor de código. Aqui está o código: Você é um codificador e precisa escrever um código Python para análise temporal de vendas no arquivo Walmart.csv . Faça a leitura e escrita em um arquivo.
             ^
SyntaxError: invalid syntax

Recompensa Codificador: 0
Reco

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# Teste do modelo

In [ ]:
import matplotlib.pyplot as plt

# Lista para armazenar recompensas ao longo do tempo
rewards_history = []

# Testar o modelo treinado
obs = env.reset()
for i in range(500):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)

    rewards_history.append(rewards)  # Armazena a recompensa do episódio atual

    env.render()

    if done:
        obs = env.reset()

env.close()

# Gerar o gráfico de recompensas
plt.plot(rewards_history)
plt.xlabel("Episódios")
plt.ylabel("Recompensa Total")
plt.title("Evolução das Recompensas ao Longo do Tempo")
plt.show()

# Para carregar o modelo mais tarde
# model = PPO.load("ppo_data_analysis")

# Visualizando dados de métricas de treinamento

In [ ]:
import pandas as pd

# Carregar o log CSV gerado
log_data = pd.read_csv('/content/logs/progress.csv')

# Visualizar as primeiras linhas
print(log_data.head(20))